# Setup

## Clone

In [ ]:
!git clone https://github.com/noamdwc/grpo_chess.git

Cloning into 'grpo_chess'...
remote: Enumerating objects: 715, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 715 (delta 176), reused 243 (delta 107), pack-reused 358 (from 1)
Receiving objects: 100% (715/715), 376.74 KiB | 22.16 MiB/s, done.
Resolving deltas: 100% (424/424), done.


In [ ]:
!ls src

ls: cannot access 'src': No such file or directory


In [ ]:
import sys
REPO_PATH = '/content/grpo_chess'
sys.path.append(REPO_PATH)
%cd {REPO_PATH}

/content/grpo_chess


In [ ]:
!git checkout self_play

Branch 'self_play' set up to track remote branch 'self_play' from 'origin'.
Switched to a new branch 'self_play'


In [ ]:
%pip install -q pytorch_lightning wandb python-chess jaxtyping datasets huggingface_hub # The tokenization from searchless_chess require jaxtyping

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 108.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 47.6 MB/s eta 0:00:00


In [ ]:
%cd src
!python ../pull_searchelss_chess_model.py
%cd ..

/content/grpo_chess/src
Fetching 19 files:   0% 0/19 [00:00<?, ?it/s]
.gitattributes: 1.52kB [00:00, 4.15MB/s]
Fetching 19 files:   5% 1/19 [00:00<00:05,  3.18it/s]
README.md: 5.16kB [00:00, 14.7MB/s]

config.json: 100% 187/187 [00:00<00:00, 1.97MB/s]
Fetching 19 files:  16% 3/19 [00:00<00:02,  6.83it/s]
__init__.py: 100% 31.0/31.0 [00:00<00:00, 428kB/s]

config.cpython-310.pyc: 100% 1.83k/1.83k [00:00<00:00, 21.1MB/s]

model_info.json: 100% 233/233 [00:00<00:00, 2.89MB/s]
Fetching 19 files:  21% 4/19 [00:00<00:02,  7.06it/s]
constants.cpython-310.pyc: 100% 3.57k/3.57k [00:00<00:00, 44.1MB/s]

hf_model.cpython-310.pyc: 100% 9.00k/9.00k [00:00<00:00, 57.2MB/s]

params.npz:   0% 0.00/35.8M [00:00<?, ?B/s]

tokenizer.cpython-310.pyc: 100% 2.22k/2.22k [00:00<00:00, 19.3MB/s]


transformer.cpython-310.pyc: 100% 7.80k/7.80k [00:00<00:00, 64.2MB/s]


utils.cpython-310.pyc: 100% 4.20k/4.20k [00:00<00:00, 31.9MB/s]


constants.py: 3.34kB [00:00, 19.7MB/s]


hf_model.py: 11.6kB [00:00, 37.9MB/s]

## Reload

Since `%autoreload` is unavailable, you can use `importlib.reload` to update modules after modifying their code.

In [ ]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 530 bytes | 265.00 KiB/s, done.
From https://github.com/noamdwc/grpo_chess
   a50bca9..c349968  self_play  -> origin/self_play
Updating a50bca9..c349968
Fast-forward
 src/grpo_logic/model.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
import importlib
import src

# # After modifying the code in the file, run this to reload the module:
importlib.reload(src)
# # If you use 'from ... import ...', you may need to re-run the import line as well:

<module 'src' from '/content/grpo_chess/src/__init__.py'>

## After Session Restart

In [ ]:
import sys
REPO_PATH = '/content/grpo_chess'
sys.path.append(REPO_PATH)
%cd {REPO_PATH}

/content/grpo_chess


# Run code

## Train Script

In [ ]:
!wandb login $(cat ../drive/MyDrive/wandb/key.txt)

wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
!apt-get -qq install -y stockfish # Download stockfish

In [ ]:
from torch.utils.data import DataLoader

from src.trainer import get_trainer
from src.chess.boards_dataset import ChessStartStatesDataset
from src.grpo_logic.model import GRPOChessTransformer
from src.configs.config_loader import load_experiment_config

In [ ]:
DEFALT_CONFIG_PATH = '/content/grpo_chess/src/configs/default.yaml'
OVERRIDE_CONFIGS = {'pretrain': {'checkpoint_path': '/content/drive/MyDrive/data/grpo-chess/pretrain_checkpoints/pretrain-20260128-1215-16es-epoch=12-train/loss=2.3285.ckpt'}}
config = load_experiment_config(DEFALT_CONFIG_PATH, overrides=OVERRIDE_CONFIGS)
dataloader_config = {
        "batch_size": config.training.batch_size,
        "num_workers": 2,
    }

In [ ]:
trainer = get_trainer(num_epochs=config.training.num_epochs)
dataset = ChessStartStatesDataset(config.dataset)
dataloader = DataLoader(dataset, **dataloader_config)
model = GRPOChessTransformer(
        transformer_config=config.transformer,
        grpo_config=config.grpo,
        eval_cfg=config.eval,
        stockfish_cfg=config.stockfish,
        policy_cfg=config.policy,
        searcher_cfg=config.searcher,
        pretrain_cfg=config.pretrain)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


Generated run name: chess-grpo-20260128-1822-xgsr
Loading pretrained weights from: /content/drive/MyDrive/data/grpo-chess/pretrain_checkpoints/pretrain-20260128-1215-16es-epoch=12-train/loss=2.3285.ckpt


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL src.models.ChessTransformerConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([src.models.ChessTransformerConfig])` or the `torch.serialization.safe_globals([src.models.ChessTransformerConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.lo

In [ ]:
print("Starting Training with WandB Tracking...")
try:
  trainer.fit(model, dataloader)
except Exception as e:
  print(e)
  with open('/content/drive/MyDrive/data/grpo-chess/error.txt', 'w') as f:
    f.write(str(e))
  raise e

## Pretrain

In [ ]:
PRETRAIN_DATASET_CACHE_DIR = '/content/drive/MyDrive/data/grpo-chess/pretrain_processed_dataset_5m'
OVERRIDE_CONFIGS = {'dataset': {'cache_path': PRETRAIN_DATASET_CACHE_DIR, 'max_samples': int(5e6)},
                    'pretrain': {'checkpoint_dir': '/content/drive/MyDrive/data/grpo-chess/pretrain_checkpoints'}}

In [ ]:
!wandb login $(cat ../drive/MyDrive/wandb/key.txt)

wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
import time
import random
import string

from torch.utils.data import DataLoader

from src.pretrain.pretrain import (load_pretrain_config,
                                                  train,
                                                  PretrainChessTransformer,
                                                  get_pretrain_trainer,
                                                  collate_pretrain_batch,
                                                  ChessPretrainDataset,
                                                  PretrainDatasetConfig
                                                  )

In [ ]:
pretrain_config, dataset_config, transformer_config = load_pretrain_config(overrides=OVERRIDE_CONFIGS)

In [ ]:
!cp {dataset_config.cache_path}/*.pt /content/

In [ ]:
dataset_config.cache_path = '/content/'

In [ ]:
train(pretrain_config, dataset_config, transformer_config)

Epoch 11/21 ━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 405/1221 0:05:41 • 0:11:07 1.23it/s v_num: 1vv0 train/loss: 2.365     
                                                                                 train/accuracy: 0.370 val/loss:   
                                                                                 2.412 val/accuracy: 0.364         